<a href="https://colab.research.google.com/github/zangston/CS-4774/blob/master/Homework%202/CS4774_Homework_02_final_submission.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Homework 02: Linear Classification

Deadline: Monday March 13, 2023 11:59 PM (without a late penalty)

---

This homework consists two sections, which covers the Perceptron algorithm and logistic regression classifiers that we learned. 

If you are running this notebook on your local computer, please make sure it has the following packages installed

- Numpy
- Sklearn

---

**Requirements**:

Please read the following requirements carefully, to avoid any unnecessary point deduction. 

1. Keep all the required results in your submission, our TAs will grade the homework based on the submitted results. (The TAs not run the code unless they have some questions about the implementation.)
2. Your submission should only be via Canvas.

## 1 The Perceptron Algorithm (5 points)

### 1.1 Implementation (3.5 points)

Let's implement the Perceptron algorithm described in our class lecture. 

First, we need to download a toy dataset from the course webpage and load it. 

In [26]:
import urllib.request
import numpy as np
url = "https://yangfengji.net/uva-ml-undergrad/data/separable.txt"
filename, headers = urllib.request.urlretrieve(url, filename="separable.txt")

data = arr = np.loadtxt("separable.txt", delimiter="\t", dtype=float)
X = data[:,:2] # Input
Y = data[:,-1] # Label

# Attach one column to X for capturing the bias term in classification
X = np.concatenate((X, np.ones((X.shape[0], 1))), axis=1)

# X is the first array, with each point's coordinate
print(X)
# Y is the second array, which stores each point's sign
print(Y)

[[-0.11  0.59  1.  ]
 [ 0.35  0.79  1.  ]
 [ 0.25  0.46  1.  ]
 [ 0.03  0.26  1.  ]
 [-0.37  0.38  1.  ]
 [ 0.25  0.12  1.  ]
 [-0.06 -0.03  1.  ]
 [ 0.2  -0.2   1.  ]
 [ 0.55  0.08  1.  ]
 [ 0.81 -0.11  1.  ]
 [ 0.54 -0.23  1.  ]
 [-0.11 -0.23  1.  ]
 [ 0.14  1.03  1.  ]
 [ 0.59  1.21  1.  ]]
[-1. -1. -1. -1. -1.  1.  1.  1.  1.  1.  1.  1. -1. -1.]


Please implement the Perceptron algorithm in the following code block.

Make sure your implementation has the following components

- The Percpetron updating rule and the condition of when to run the updating rule;
- The convergence criterion -- stop training when the classifier can make correct prediction of all the training examples;
- Print out the classification weight in the end. 

In [51]:
def perceptron(X, Y):
    # initialize weight vector and bias term
    weights = np.zeros(3)

    # Set convergence criterion to false before execution
    convergence = 0

    while convergence == 0:
        '''
        We assume the classifier is true, and then we test its correctness
            If assumption of correctness is true, loop ends
            If not, we set assumption to false, which repeats the loop
        '''
        convergence = 1

        '''
        The notes kind of confused me with the i <- t % m line but basically 
            it looks like the intent is to be able wrap around the training set
            to continually update the weights while the algorithm is still
            training.
        Using a while loop with a nested for loop implicitly uses this technique
            without needing to declare separate t and i variables
        '''
        for i in range(len(X)):
            if Y[i] * np.dot(weights, X[i]) <= 0:
                weights += X[i] * Y[i]
                convergence = 0
            # print(weights)
        # print(weights)

    return weights

print("Classification weights:" , perceptron(X, Y))


Classification weights: [ 2.22 -4.17  1.  ]


### 1.2 The Difference between Perceptron and Logisitic Regression (1.5 points)

Please list the major difference between the Perceptron model and the Logistic Regression model as we discussed in class. Please list at least three of them.

**Answer**

1.   The perceptron model uses an iterative algorithm that updates the classification weights until the convergence criterion is satisfied. The logistic regression model uses gradient-based learning in its optimization of its predictions.
2. The perceptron algorithm's updating rule also only applies when a prediction is wrong. This is not the same for logistic regression, whose updating rule also applies to correction predictions
3.   The logistic regression model defines the probability of assignment of label *y*, based on the given input *x*. The perceptron model does not have the same probabilistic interpretation. 
4. Online learning vs. batch learning: the perceptron model is an instance of online learning, where the weights are updated based on the observed error for each subsequent data point. The logstic regression model uses batch learning, which is when a model makes a prediction for all data points, and updates based on the average error until convergence.
5. Because the logistic regression uses gradient-based learning, it does not utilize a learning rate in updating its classification weights. The perceptron does, however.



## 2 Logistic Regression (9 points)

Logistic regression can handle both linear separable and unseparable cases. Therefore, we will use a real-world example datasets, instead of a synthetical one. 

We will use the [Logistic Regression]() model from Sklearn for the questions. In other words, you don't have to implement a logistic regression model by yourself, but you need to understand how to use it and how to interpret its outputs. 

### 2.0 Download the dataset

Run the following code and download the [xxx]() dataset from [OpenML](). Similar to what we explained in class, we will also use the [OridinalEncoder]() to convert the non-numeric features into numeric features. 

Furthermore, we will run the data split function to divide the whole dataset into a training set and a validation set. 

In [20]:
from sklearn.datasets import fetch_openml
from sklearn.model_selection import train_test_split

X, Y = fetch_openml("diabetes", version=1, return_X_y=True)
print("Read {} examples with {} features".format(X.shape[0], X.shape[1]))

print(X)

trnX, valX, trnY, valY = train_test_split(X, Y, test_size=0.3, random_state=111)
print("The size of training set: {}".format(trnX.shape[0]))
print("The size of validation set: {}".format(valX.shape[0]))

/usr/local/lib/python3.9/dist-packages/sklearn/datasets/_openml.py:968: FutureWarning: The default value of `parser` will change from `'liac-arff'` to `'auto'` in 1.4. You can set `parser='auto'` to silence this warning. Therefore, an `ImportError` will be raised from 1.4 if the dataset is dense and pandas is not installed. Note that the pandas parser may return different data types. See the Notes Section in fetch_openml's API doc for details.
  warn(


Read 768 examples with 8 features
     preg   plas  pres  skin   insu  mass   pedi   age
0     6.0  148.0  72.0  35.0    0.0  33.6  0.627  50.0
1     1.0   85.0  66.0  29.0    0.0  26.6  0.351  31.0
2     8.0  183.0  64.0   0.0    0.0  23.3  0.672  32.0
3     1.0   89.0  66.0  23.0   94.0  28.1  0.167  21.0
4     0.0  137.0  40.0  35.0  168.0  43.1  2.288  33.0
..    ...    ...   ...   ...    ...   ...    ...   ...
763  10.0  101.0  76.0  48.0  180.0  32.9  0.171  63.0
764   2.0  122.0  70.0  27.0    0.0  36.8  0.340  27.0
765   5.0  121.0  72.0  23.0  112.0  26.2  0.245  30.0
766   1.0  126.0  60.0   0.0    0.0  30.1  0.349  47.0
767   1.0   93.0  70.0  31.0    0.0  30.4  0.315  23.0

[768 rows x 8 columns]
The size of training set: 537
The size of validation set: 231


### 2.1 Build the Classifier (2 points)

Please follow the instructions and complete the code block for building an LR classifier. 

- Use the [Logistic Regression](https://scikit-learn.org/stable/modules/generated/sklearn.linear_model.LogisticRegression.html) model from Sklearn with its **default** parameters.
- Training the model and report its prediction accuracy on **both** the training and the validation sets.
- Keep the **printed** (instead of hand typing) results in your submission for grading. Make sure the printed results have enough information about which is training accuracy and which one is validation accuracy. For example, you can use 
```python
print("Training accuracy: {}".format(trn_acc))
```

In [35]:
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score

'''
initialize model and fit to data
i was getting a nonconvergence error and the error message recommended i try 
    increasing the number of iterations. please god i hope it works
IT WORKS!!!
'''
model = LogisticRegression(max_iter=1000)
model.fit(trnX, trnY)

# generate predictions
predict_trnY = model.predict(trnX)
predict_valY = model.predict(valX)

# compute accuracy
trn_acc = accuracy_score(trnY, predict_trnY)
val_acc = accuracy_score(valY, predict_valY)

# output results
print("Training accuracy: {}".format(trn_acc))
print("Validation accuracy : {}".format(val_acc))


Training accuracy: 0.7802607076350093
Validation accuracy : 0.7532467532467533


### 2.2 Feature Pre-processing (2 points)

In our lecture, we talk about three different ways to represent the features before feeding into the classifier. 

Pick the [OneHotEncoder](https://scikit-learn.org/stable/modules/generated/sklearn.preprocessing.OneHotEncoder.html#sklearn.preprocessing.OneHotEncoder) to further processing the features, and building the classifier again. 

Please follow the instructions and complete the code block for building an LR classifier with feature pre-processing

- Pick one pre-processing method to further scale or encode the features
- Training the model and report its prediction accuracy on **both** the training and the validation sets.
- Keep the **printed** (instead of hand typing) results in your submission for grading. Make sure the printed results have enough information about which is training accuracy and which one is validation accuracy.

Note that, you can find many other pre-processing functions in Sklearn via [this link](https://scikit-learn.org/stable/modules/classes.html#module-sklearn.preprocessing). Since we only discussed the three abovementioned pre-processing methods in class, you are *not* required to do anything beyond the instruction. 

In [39]:
from sklearn.preprocessing import OneHotEncoder

# initialize encoder, train on both sets
oneHot = OneHotEncoder(handle_unknown='ignore')
oneHot.fit(trnX)
trnOneHot = oneHot.transform(trnX)
valOneHot = oneHot.transform(valX)

# fit logistic regression to data
model = LogisticRegression(max_iter=1000)
model.fit(trnOneHot, trnY)

# generate predictions
predict_trnY = model.predict(trnOneHot)
predict_valY = model.predict(valOneHot)

# calculate accuracies
trn_acc = accuracy_score(trnY, predict_trnY)
val_acc = accuracy_score(valY, predict_valY)

# print results
print("Training Accuracy: ", trn_acc)
print("Validation Accuracy: ", val_acc)


Training Accuracy:  0.9739292364990689
Validation Accuracy:  0.7142857142857143


### 2.3 Justification of Pre-processing (2 points)

If your implementation is correct, you should see a significant boost on both training and validation accuracies. 

Based on what we discussed in class, please explain why this one-hot encoding is helpful for improving classification performance.

**Answer**

One-hot encoding allows categorical variables to be split into their own boolean vector "dummy" variables, for each category. This formats categorical data that is more easily interpreted by the learning algorithm and allows the  model to better understand the effect of any particular category being "true", all else equal.

This technique is also applied when creating regression models in statistical and econometric contexts.

### 2.4 $\ell_2$ Regularization (1.5 points)

With the pre-processed data, please train the classifiers with three different regularization coefficient. In the [Logistic Regression]() model implemented in Sklearn, the associated function argument is $C$. 

Please use the following three code blocks and try three different $C$ values

- $C=100.0$
- $C=1.0$
- $C=0.01$

and pirnt out 

- the training accuracy, and 
- the validation accuracy 

by following the **same** format as the previous questions

$C=100.0$

In [40]:
# fit logistic regression to data
model = LogisticRegression(max_iter=1000, C = 100.0)
model.fit(trnOneHot, trnY)

# generate predictions
predict_trnY = model.predict(trnOneHot)
predict_valY = model.predict(valOneHot)

# calculate accuracies
trn_acc = accuracy_score(trnY, predict_trnY)
val_acc = accuracy_score(valY, predict_valY)

# print results
print("Training Accuracy: ", trn_acc)
print("Validation Accuracy: ", val_acc)

Training Accuracy:  1.0
Validation Accuracy:  0.7229437229437229


$C=1.0$

In [41]:
# fit logistic regression to data
model = LogisticRegression(max_iter=1000, C = 1.0)
model.fit(trnOneHot, trnY)

# generate predictions
predict_trnY = model.predict(trnOneHot)
predict_valY = model.predict(valOneHot)

# calculate accuracies
trn_acc = accuracy_score(trnY, predict_trnY)
val_acc = accuracy_score(valY, predict_valY)

# print results
print("Training Accuracy: ", trn_acc)
print("Validation Accuracy: ", val_acc)

Training Accuracy:  0.9739292364990689
Validation Accuracy:  0.7142857142857143


$C=0.01$

In [42]:
# fit logistic regression to data
model = LogisticRegression(max_iter=1000, C = 0.01)
model.fit(trnOneHot, trnY)

# generate predictions
predict_trnY = model.predict(trnOneHot)
predict_valY = model.predict(valOneHot)

# calculate accuracies
trn_acc = accuracy_score(trnY, predict_trnY)
val_acc = accuracy_score(valY, predict_valY)

# print results
print("Training Accuracy: ", trn_acc)
print("Validation Accuracy: ", val_acc)

Training Accuracy:  0.6443202979515829
Validation Accuracy:  0.6666666666666666


### 2.5 Explan the regularization effect (1.5 points)

Based on the classification accuracies above, explain the $\ell_2$ regularization effect for the classification performance

- What is the direct effect of $\ell_2$ regularization?
- Does it work all the time or when it may work?
- How do we know it works for helping avoid over-fitting?

Make sure your answer covers all the three questions above.

**Answer**

ℓ2 regularization changes the loss function of the logistic regression model in a way that penalizes higher complexity in a model. This encourages the learning model to estimate a simpler fit.

ℓ2 regularization works in situations where a dataset has many features, some of which might be irrelevant. The modified loss function encourages the learning model to attempt to "drop" certain categorical variables, driving their weights to zero. Inversely, ℓ2 will hurt the accuracy of a model if there are many equally important features, as it penalizes higher weights. If all weights are equally important, the model will naturally be complex, which is what ℓ2 sets out to prevent.

The primary goal of ℓ2 regularization is to punish higher weight assignments. Overfitting occurs when a model is too complex, and the regression is too closely fitted to a dataset, losing efficacy in light of newly introduced data. Thus regularization, encouraging a simpler model via smaller weights, preserves a model's general goodness-of-fit, which prevents overfitting.